# Operaciones sobre RDDs

## Algunos datos de ejemplo

In [1]:
# Se borra la carpeta y su contenido en la máquina local si existe
!rm -rf wordcount/

# Se crea la carpeta vacía
!mkdir -p wordcount/

In [100]:
%%writefile wordcount/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns
in data. Especially valuable in areas rich with recorded information, analytics relies
on the simultaneous application of statistics, computer programming and operations research
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business
performance. Specifically, areas within analytics include predictive analytics, prescriptive
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization,
marketing optimization and marketing mix modeling, web analytics, call analytics, speech
analytics, sales force sizing and optimization, price and promotion modeling, predictive
science, credit risk analysis, and fraud analytics. Since analytics can require extensive
computation (see big data), the algorithms and software used for analytics harness the most
current methods in computer science, statistics, and mathematics.

Overwriting wordcount/text0.txt


In [3]:
%%writefile wordcount/text1.txt
The field of data analysis. Analytics often involves studying past historical data to
research potential trends, to analyze the effects of certain decisions or events, or to
evaluate the performance of a given tool or scenario. The goal of analytics is to improve
the business by gaining knowledge which can be used to make improvements or changes.

Writing wordcount/text1.txt


In [4]:
%%writefile wordcount/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions
about the information they contain, increasingly with the aid of specialized systems
and software. Data analytics technologies and techniques are widely used in commercial
industries to enable organizations to make more-informed business decisions and by
scientists and researchers to verify or disprove scientific models, theories and
hypotheses.

Writing wordcount/text2.txt


In [5]:
# crea la carpeta /tmp/wordcount en el HDFS
!hdfs dfs -mkdir /tmp/wordcount

# copia los archvios del directorio local wordcount/
# al directorio /tmp/wordcount/ en el hdfs
!hdfs dfs -copyFromLocal wordcount/* /tmp/wordcount/

## Preparamos el entorno de ejecución

In [63]:
#
# findspark permite usar pyspark (interfaz de Python a Spark),
# desde cualquier programa escrito en Python.
#
import findspark

findspark.init()

#
# A continuación se inicializan las variables obligatorias
# requeridas para trabajar con Spark desde Python:
#
#  SparkContext representa la conexión al cluster de Spark.
#  SparkConf representa la configuración particular de una aplicación
#     escrita en Spark.
#  SparkSession representa la conexión para trabajar con SQL.
#
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

sparkConf = SparkConf().setAppName("My SparkQL Application")
# sc.stop()
sc = SparkContext(conf=sparkConf)
sc.setLogLevel("WARN")
spark = SparkSession(sc)

22/11/21 15:44:51 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Trabajamos con RDDs

In [66]:
#
# Creamos un RDD a partir de una lista
#
rdd = sc.parallelize([1, 2, 3, 4, 5])

#
# Un RDD es un objeto en memoria que gestiona la lista, no la lista como tal
#
rdd

ParallelCollectionRDD[2] at readRDDFromFile at PythonRDD.scala:274

In [67]:
#
# Para ver la lista usamos collect()
#
rdd.collect()

[1, 2, 3, 4, 5]

In [68]:
#
# Creamos un RDD a partir de un diccionario
#
rdd = sc.parallelize({"a": 1, "b": 2, "c": 3, "d": 4})
rdd.collect()

['a', 'b', 'c', 'd']

In [71]:
#
# Creamos una secuencia de parejas clave-valor. Empezamos por las claves
#
rdd = sc.parallelize([1, 2, 3, 4, 5])

#
# Una vez se tiene un objeto RDD, es posible aplicar operaciones
# en paralelo sobre él. En la siguiente línea de código, se aplica
# la función lambda x:(x, x**2) a cada elemento del RDD, mediante
# la función map. Como resultado se genera una secuencia de pares
# [(1, 1), (2, 4), ..., (5, 25)]
#
rdd = rdd.map(lambda x: (x, x**2))

#
# Guardamos la secuencia en HDFS, en formato "SequenceFile"
#
rdd.saveAsSequenceFile("/tmp/out")

In [72]:
#
# El archivo se almacena en /tmp/out/. Aquí se usa
# ls para listar los archivos de la carpeta /tmp/ del HDFS
# Como resultado de la operación anterior, se ha creado la carpeta
# /tmp/out
#
!hdfs dfs -ls /tmp/out/

Found 3 items
-rw-r--r--   1 root supergroup          0 2022-11-21 15:51 /tmp/out/_SUCCESS
-rw-r--r--   1 root supergroup        124 2022-11-21 15:51 /tmp/out/part-00000
-rw-r--r--   1 root supergroup        140 2022-11-21 15:51 /tmp/out/part-00001


In [74]:
#
# Una vez se tiene una secuencia de pares almacenada en el hdfs,
# es posible volverla a cargar mediante sequenceFile().
#
sc.sequenceFile("/tmp/out").collect()

[(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]

In [75]:
#
# Limpiamos el HDFS
#
!hdfs dfs -rm -r -f /tmp/out

Deleted /tmp/out


## Trabajo con textFile

In [77]:
#
# textFile() puede leer un archivo de texto o los archivos
# especificados. A continuación, se leen todos los archivos
# de texto de la carpeta /tmp/wordcount/ en el hdfs
#
rdd = sc.textFile("/tmp/wordcount/")
rdd

/tmp/wordcount/ MapPartitionsRDD[18] at textFile at NativeMethodAccessorImpl.java:0

In [78]:
#
# Se obtiene el timpo de dato almacenado en la variable rdd
#
type(rdd)

pyspark.rdd.RDD

In [79]:
#
# rdd almacena como strings, el contenido de los archivos
# text0.txt, text1.txt y text2.txt.
# Se combinan todos los ficheros .txt, el resultado es una lista de registros de una línea cada uno
#
rdd.collect()

['Analytics is the discovery, interpretation, and communication of meaningful patterns',
 'in data. Especially valuable in areas rich with recorded information, analytics relies',
 'on the simultaneous application of statistics, computer programming and operations research',
 'to quantify performance.',
 '',
 'Organizations may apply analytics to business data to describe, predict, and improve business',
 'performance. Specifically, areas within analytics include predictive analytics, prescriptive',
 'analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big',
 'Data Analytics, retail analytics, store assortment and stock-keeping unit optimization,',
 'marketing optimization and marketing mix modeling, web analytics, call analytics, speech',
 'analytics, sales force sizing and optimization, price and promotion modeling, predictive',
 'science, credit risk analysis, and fraud analytics. Since analytics can require extensive',
 'computation (see big data)

In [80]:
#
# A diferencia de TextFile(), que combina varios ficheros en uno y lee por líneas,
# wholeTextFiles() devuelve tantos registros como ficheros, y cada registro contiene
# todo el texto de cada uno de los ficheros. En este caso, 3 ficheros --> tres registros <clave, valor>
# donde clave=identificador del fichero, valor=contenido.
#
rdd = sc.wholeTextFiles("/tmp/wordcount/")
for row in rdd.collect():
    print(row)
    print("----")

('hdfs://0.0.0.0:9000/tmp/wordcount/text0.txt', 'Analytics is the discovery, interpretation, and communication of meaningful patterns\nin data. Especially valuable in areas rich with recorded information, analytics relies\non the simultaneous application of statistics, computer programming and operations research\nto quantify performance.\n\nOrganizations may apply analytics to business data to describe, predict, and improve business\nperformance. Specifically, areas within analytics include predictive analytics, prescriptive\nanalytics, enterprise decision management, descriptive analytics, cognitive analytics, Big\nData Analytics, retail analytics, store assortment and stock-keeping unit optimization,\nmarketing optimization and marketing mix modeling, web analytics, call analytics, speech\nanalytics, sales force sizing and optimization, price and promotion modeling, predictive\nscience, credit risk analysis, and fraud analytics. Since analytics can require extensive\ncomputation (se

In [82]:
#
# Carga de los archivos.
#
rdd = sc.textFile("/tmp/wordcount/")

#
# Calculo de la longitud de cada línea usando la función len().
# Se aplica la función len() a cada elemento del rdd. Como se ha leído con textFile()
# se aplica a cada línea del texto
#
rdd = rdd.map(len)
print(rdd.collect())

#
# Cálculo del total de caracteres, haciendo una reducción sobre la lista de longitudes
#
rdd = rdd.reduce(lambda a, b: a + b)
rdd

[84, 86, 91, 24, 0, 93, 92, 90, 87, 88, 88, 89, 91, 65, 85, 87, 89, 84, 86, 84, 86, 82, 80, 11]


1842

In [83]:
#
# En este ejemplo se pasa una función arbitraria a `map`.
# La función add es equivalente a `lambda a, b: a+b`
# Del manual de Python: "The operator module exports a set
# of efficient functions corresponding to the intrinsic operators of Python"
#

from operator import add

rdd = sc.textFile("/tmp/wordcount/")
rdd = rdd.map(len)
print(rdd.collect())
rdd = rdd.reduce(add)
rdd

[84, 86, 91, 24, 0, 93, 92, 90, 87, 88, 88, 89, 91, 65, 85, 87, 89, 84, 86, 84, 86, 82, 80, 11]


1842

In [88]:
#
# creación de parejas (key, value). Se desea
# contar el número de ocurrencias de cada letra
# en la siguiente lista.
#
rdd = sc.parallelize(["a", "b", "a", "c", "d", "a", "b"])

##
# Se crean las parejas <clave, valor>
##
rdd = rdd.map(lambda s: (s, 1))
rdd.collect()

[('a', 1), ('b', 1), ('a', 1), ('c', 1), ('d', 1), ('a', 1), ('b', 1)]

In [89]:
#
# Se reducen las parejas con la misma clave, sumando
# los valores.
#
rdd = rdd.reduceByKey(lambda a, b: a + b)
print(rdd.collect())

[('b', 2), ('c', 1), ('d', 1), ('a', 3)]


In [107]:
#
# Generamos un fichero "data.txt" con líneas con registros, separados por espacios
#

!rm data.txt

In [108]:
%%writefile data.txt
E   1       b,g,f   jjj:3,bbb:0,ddd:9,ggg:8,hhh:2
A   2       a,f,c   ccc:2,ddd:0,aaa:3,hhh:9
B   5       f,e,a,c ddd:2,ggg:5,ccc:6,jjj:1
A   3       a,b     hhh:9,iii:5,eee:7,bbb:1
C   6       f,g,d,a iii:6,ddd:5,eee:4,jjj:3
A   7       c,d     bbb:2,hhh:0,ccc:4,fff:1,aaa:7
A   9       g,d,a   aaa:5,fff:8,ddd:2,iii:0,jjj:7,ccc:1
B   1       b,a     fff:3,hhh:1,ddd:2
E   2       d,e,a,f eee:4,ccc:5,iii:9,fff:7,ggg:6,bbb:0
B   3       d,b,g,f bbb:7,jjj:9,fff:5,iii:4,ggg:2,eee:3
C   7       d,c,f,b hhh:6,eee:4,iii:0,fff:2,jjj:1
C   5       d,e,a,c bbb:7,iii:6,ggg:9
D   3       g,e,f,b bbb:9,aaa:3,ccc:6,fff:4,eee:2
E   8       c,f     aaa:8,ddd:5,jjj:1
B   9       d,b     ccc:0,jjj:6,fff:7,ddd:3,aaa:2
D   1       f,e     ccc:0,eee:6,bbb:9,ddd:3
E   3       e,b,f   bbb:6,iii:3,hhh:5,fff:4,ggg:9,ddd:2
D   5       g,a     hhh:4,jjj:5,ccc:9
E   8       e,c,f,a ccc:1,iii:6,fff:9
E   9       e,a     bbb:9,aaa:3,fff:1
E   7       e,f     ddd:9,iii:2,aaa:4
E   3       c,b,g   ccc:5,fff:8,iii:7
D   5       c,f,a   eee:3,jjj:2,ddd:7
A   1       f,a,d   jjj:1,ggg:0,ccc:7,ddd:9,bbb:3
E   4       c,d     jjj:6,ccc:0,aaa:1,hhh:9,iii:7,ggg:8
E   6       e,d,c   fff:3,eee:6,iii:4,bbb:7,ddd:0,ccc:1
A   8       a,e,f   fff:0,ddd:5,ccc:4
E   5       c,a,g   ggg:6,hhh:3,ddd:9,ccc:0,jjj:7
A   6       f,e     hhh:6,jjj:0,eee:5,iii:7,ccc:3
C   0       f,c,a,g eee:1,fff:4,aaa:2,ccc:7,ggg:0,ddd:6
A   1       b,f     ccc:6,aaa:9,eee:5,ddd:0,bbb:3
D   2       b,f     bbb:7,hhh:1,aaa:6,iii:4,fff:9,ddd:5
E   5       a,c     fff:3,ccc:1,ggg:2,eee:5
B   4       b,f,c   iii:7,ggg:3,ddd:0,jjj:8,hhh:5,ccc:1
B   6       f,a,e   hhh:6,ccc:3,jjj:0,bbb:8,ddd:7
D   7       a,f     aaa:0,fff:5,ddd:3
B   8       c,a     ddd:5,jjj:2,iii:7,ccc:0,bbb:4
C   9       c,a,e,f eee:0,fff:2,hhh:6
E   1       e,d     fff:9,iii:2,eee:0
E   5       f,a,d   hhh:8,ggg:3,jjj:5

Writing data.txt


In [109]:
#
# Copia el archivo del sistema local al HDFS
# 
!hdfs dfs -copyFromLocal data.txt /tmp/data.txt

copyFromLocal: `/tmp/data.txt': File exists


In [113]:
# 
# carga los datos a un RDD desde HDFS
#
rdd = sc.textFile("/tmp/data.txt")

# separa los datos por los espacios
rdd = rdd.map(lambda x: x.split())
rdd.collect()

[['E', '1', 'b,g,f', 'jjj:3,bbb:0,ddd:9,ggg:8,hhh:2'],
 ['A', '2', 'a,f,c', 'ccc:2,ddd:0,aaa:3,hhh:9'],
 ['B', '5', 'f,e,a,c', 'ddd:2,ggg:5,ccc:6,jjj:1'],
 ['A', '3', 'a,b', 'hhh:9,iii:5,eee:7,bbb:1'],
 ['C', '6', 'f,g,d,a', 'iii:6,ddd:5,eee:4,jjj:3'],
 ['A', '7', 'c,d', 'bbb:2,hhh:0,ccc:4,fff:1,aaa:7'],
 ['A', '9', 'g,d,a', 'aaa:5,fff:8,ddd:2,iii:0,jjj:7,ccc:1'],
 ['B', '1', 'b,a', 'fff:3,hhh:1,ddd:2'],
 ['E', '2', 'd,e,a,f', 'eee:4,ccc:5,iii:9,fff:7,ggg:6,bbb:0'],
 ['B', '3', 'd,b,g,f', 'bbb:7,jjj:9,fff:5,iii:4,ggg:2,eee:3'],
 ['C', '7', 'd,c,f,b', 'hhh:6,eee:4,iii:0,fff:2,jjj:1'],
 ['C', '5', 'd,e,a,c', 'bbb:7,iii:6,ggg:9'],
 ['D', '3', 'g,e,f,b', 'bbb:9,aaa:3,ccc:6,fff:4,eee:2'],
 ['E', '8', 'c,f', 'aaa:8,ddd:5,jjj:1'],
 ['B', '9', 'd,b', 'ccc:0,jjj:6,fff:7,ddd:3,aaa:2'],
 ['D', '1', 'f,e', 'ccc:0,eee:6,bbb:9,ddd:3'],
 ['E', '3', 'e,b,f', 'bbb:6,iii:3,hhh:5,fff:4,ggg:9,ddd:2'],
 ['D', '5', 'g,a', 'hhh:4,jjj:5,ccc:9'],
 ['E', '8', 'e,c,f,a', 'ccc:1,iii:6,fff:9'],
 ['E', '9', 'e,a', 

In [114]:
# 
# extrae la segunda columna y la convierte de texto a número
# 
rdd = rdd.map(lambda x: int(x[1]))
rdd.collect()[0:5]  # se imprimen los primeros cinco valores

[1, 2, 5, 3, 6]

In [115]:
from operator import add

rdd.reduce(add)  # Se reduce sumando los valores. Note que no hay parejas clave-valor

190

In [126]:
#
# El codigo se puede reescribir como:
#
sc.textFile("/tmp/data.txt").map(lambda x: x.split()).map(lambda x: int(x[1])).reduce(add)

190

In [127]:
# Carga el archivo y lo separa por espacios
rdd = sc.textFile("/tmp/data.txt").map(lambda x: x.split())

#
# Se forman parejas <columna 2, [columna 1]>, puesto que
# se desea agrupar por la columna 2. Esto es,
# [ (1, E), (2, A), ..., (1, E), (5, E)].
#

rdd = rdd.map(lambda x: (x[1], [x[0]]))
rdd.collect()

[('1', ['E']),
 ('2', ['A']),
 ('5', ['B']),
 ('3', ['A']),
 ('6', ['C']),
 ('7', ['A']),
 ('9', ['A']),
 ('1', ['B']),
 ('2', ['E']),
 ('3', ['B']),
 ('7', ['C']),
 ('5', ['C']),
 ('3', ['D']),
 ('8', ['E']),
 ('9', ['B']),
 ('1', ['D']),
 ('3', ['E']),
 ('5', ['D']),
 ('8', ['E']),
 ('9', ['E']),
 ('7', ['E']),
 ('3', ['E']),
 ('5', ['D']),
 ('1', ['A']),
 ('4', ['E']),
 ('6', ['E']),
 ('8', ['A']),
 ('5', ['E']),
 ('6', ['A']),
 ('0', ['C']),
 ('1', ['A']),
 ('2', ['D']),
 ('5', ['E']),
 ('4', ['B']),
 ('6', ['B']),
 ('7', ['D']),
 ('8', ['B']),
 ('9', ['C']),
 ('1', ['E']),
 ('5', ['E'])]

In [128]:
# 
# Ahora agrupamos por la clave numérica. El 
# resultado es una colección <clave, valor> en el que cada valor es una lista
#
rdd.reduceByKey(add).collect()

[('1', ['E', 'B', 'D', 'A', 'A', 'E']),
 ('9', ['A', 'B', 'E', 'C']),
 ('8', ['E', 'E', 'A', 'B']),
 ('4', ['E', 'B']),
 ('0', ['C']),
 ('2', ['A', 'E', 'D']),
 ('5', ['B', 'C', 'D', 'D', 'E', 'E', 'E']),
 ('3', ['A', 'B', 'D', 'E', 'E']),
 ('6', ['C', 'E', 'A', 'B']),
 ('7', ['A', 'C', 'E', 'D'])]

In [132]:
# Carga el archivo y lo separa por espacios
rdd = sc.textFile("/tmp/data.txt").map(lambda x: x.split())

#
# Se obtiene un rdd que contenga únicamente la columna 4.
#
#  ['jjj:3,bbb:0,ddd:9,ggg:8,hhh:2',
#  ...
#  'hhh:8,ggg:3,jjj:5']
#
rdd = rdd.map(lambda x: x[3])

#
# Se parte por la ','. flatMap() genera un nuevo registro
# en el rdd por cada valor en la lista retornada por
# la función lambda x: x.split(',')
#
#   ['jjj:3',
#    'bbb:0',
#    ......
#    'ggg:3',
#    'jjj:5']
#
rdd = rdd.flatMap(lambda x: x.split(","))

#
# Ahora separamos por los ':' y tomamos el primer elemento.
#
#   ['jjj',
#    'bbb',
#    .....
#    'ggg',
#    'jjj']
#
rdd = rdd.map(lambda x: x.split(":")[0])
rdd.collect()[0:10]

['jjj', 'bbb', 'ddd', 'ggg', 'hhh', 'ccc', 'ddd', 'aaa', 'hhh', 'ddd']

In [133]:
#
# Se aplica el algoritmo de conteo de palabras
#
rdd = rdd.map(lambda s: (s, 1)).reduceByKey(add)
rdd.collect()

[('jjj', 18),
 ('ccc', 23),
 ('aaa', 13),
 ('iii', 18),
 ('eee', 15),
 ('bbb', 16),
 ('ddd', 23),
 ('ggg', 13),
 ('hhh', 16),
 ('fff', 20)]

In [135]:
# Carga el archivo y lo separa por espacios
rdd = sc.textFile("/tmp/data.txt").map(lambda x: x.split())

#
# Elimina la columna 2. Queda así:
#
#   [['E', 'b,g,f', 'jjj:3,bbb:0,ddd:9,ggg:8,hhh:2'],
#    ['A', 'a,f,c', 'ccc:2,ddd:0,aaa:3,hhh:9'],
#   ....
#   ['E', 'e,d', 'fff:9,iii:2,eee:0'],
#   ['E', 'f,a,d', 'hhh:8,ggg:3,jjj:5']]
#
rdd = rdd.map(lambda x: [x[0], x[2], x[3]])

#
# Parte las columnas 2 y 3 por la coma y cuenta los elementos
#
rdd = rdd.map(lambda x: [x[0], len(x[1].split(",")), len(x[2].split(","))])

rdd.collect()

[['E', 3, 5],
 ['A', 3, 4],
 ['B', 4, 4],
 ['A', 2, 4],
 ['C', 4, 4],
 ['A', 2, 5],
 ['A', 3, 6],
 ['B', 2, 3],
 ['E', 4, 6],
 ['B', 4, 6],
 ['C', 4, 5],
 ['C', 4, 3],
 ['D', 4, 5],
 ['E', 2, 3],
 ['B', 2, 5],
 ['D', 2, 4],
 ['E', 3, 6],
 ['D', 2, 3],
 ['E', 4, 3],
 ['E', 2, 3],
 ['E', 2, 3],
 ['E', 3, 3],
 ['D', 3, 3],
 ['A', 3, 5],
 ['E', 2, 6],
 ['E', 3, 6],
 ['A', 3, 3],
 ['E', 3, 5],
 ['A', 2, 5],
 ['C', 4, 6],
 ['A', 2, 5],
 ['D', 2, 6],
 ['E', 2, 4],
 ['B', 3, 6],
 ['B', 3, 5],
 ['D', 2, 3],
 ['B', 2, 5],
 ['C', 4, 3],
 ['E', 2, 3],
 ['E', 3, 3]]

In [136]:
# Carga el archivo y lo separa por espacios
rdd = sc.textFile("/tmp/data.txt").map(lambda x: x.split())

#
# Elimina las dos primeras columnas.
#
# Resultado:
#
#  [['b,g,f', 'jjj:3,bbb:0,ddd:9,ggg:8,hhh:2'],
#   ['a,f,c', 'ccc:2,ddd:0,aaa:3,hhh:9'],
#   ...
#   ['e,d', 'fff:9,iii:2,eee:0'],
#   ['f,a,d', 'hhh:8,ggg:3,jjj:5']]
#
rdd = rdd.map(lambda x: [x[2], x[3]])

#
# Expande el dataset generando un registro por cada
# elemento de la columna 1.
#
# Resultado:
#
#  [['b', 'jjj:3,bbb:0,ddd:9,ggg:8,hhh:2'],
#   ['g', 'jjj:3,bbb:0,ddd:9,ggg:8,hhh:2'],
#  ...
#   ['a', 'hhh:8,ggg:3,jjj:5'],
#   ['d', 'hhh:8,ggg:3,jjj:5']]
#
rdd = rdd.flatMap(lambda x: [[e, x[1]] for e in x[0].split(",")])

#
# Cambia la segunda columna por una lista
# partiendo los elementos por las comas
##
# Resultado:
#
#  [['b', ['jjj:3', 'bbb:0', 'ddd:9', 'ggg:8', 'hhh:2']],
#   ['g', ['jjj:3', 'bbb:0', 'ddd:9', 'ggg:8', 'hhh:2']],
#   ....
#   ['a', ['hhh:8', 'ggg:3', 'jjj:5']],
#   ['d', ['hhh:8', 'ggg:3', 'jjj:5']]]
#
rdd = rdd.map(lambda x: [x[0], x[1].split(",")])

#
# Parte cada elemento por los ':' y toma el primer elemento
#
# Resultado:
#
#    [['b', ['jjj', 'bbb', 'ddd', 'ggg', 'hhh']],
#     ['g', ['jjj', 'bbb', 'ddd', 'ggg', 'hhh']],
#     ...
#     ['a', ['hhh', 'ggg', 'jjj']],
#     ['d', ['hhh', 'ggg', 'jjj']]]
#
rdd = rdd.map(lambda x: [x[0], [a.split(":")[0] for a in x[1]]])

#
# Genera las parejas clave valor.
#
# Resultado:
#
#   [(('b', 'jjj'), 1),
#    (('b', 'bbb'), 1),
#    ...
#    (('d', 'ggg'), 1),
#    (('d', 'jjj'), 1)]
#
rdd = rdd.flatMap(lambda x: [((x[0], a), 1) for a in x[1]])

#
# Reduce por clave para hacer el conteo.
# Se usa sortByKey() para ordenas la salida
# por claves
#
rdd.reduceByKey(add).sortByKey().collect()

[(('a', 'aaa'), 5),
 (('a', 'bbb'), 7),
 (('a', 'ccc'), 13),
 (('a', 'ddd'), 13),
 (('a', 'eee'), 7),
 (('a', 'fff'), 10),
 (('a', 'ggg'), 8),
 (('a', 'hhh'), 8),
 (('a', 'iii'), 7),
 (('a', 'jjj'), 10),
 (('b', 'aaa'), 4),
 (('b', 'bbb'), 7),
 (('b', 'ccc'), 5),
 (('b', 'ddd'), 7),
 (('b', 'eee'), 5),
 (('b', 'fff'), 8),
 (('b', 'ggg'), 4),
 (('b', 'hhh'), 7),
 (('b', 'iii'), 7),
 (('b', 'jjj'), 5),
 (('c', 'aaa'), 5),
 (('c', 'bbb'), 4),
 (('c', 'ccc'), 12),
 (('c', 'ddd'), 9),
 (('c', 'eee'), 6),
 (('c', 'fff'), 8),
 (('c', 'ggg'), 7),
 (('c', 'hhh'), 7),
 (('c', 'iii'), 8),
 (('c', 'jjj'), 8),
 (('d', 'aaa'), 4),
 (('d', 'bbb'), 6),
 (('d', 'ccc'), 7),
 (('d', 'ddd'), 5),
 (('d', 'eee'), 6),
 (('d', 'fff'), 8),
 (('d', 'ggg'), 6),
 (('d', 'hhh'), 4),
 (('d', 'iii'), 9),
 (('d', 'jjj'), 8),
 (('e', 'aaa'), 3),
 (('e', 'bbb'), 8),
 (('e', 'ccc'), 9),
 (('e', 'ddd'), 7),
 (('e', 'eee'), 7),
 (('e', 'fff'), 9),
 (('e', 'ggg'), 4),
 (('e', 'hhh'), 4),
 (('e', 'iii'), 8),
 (('e', 'jjj'),

In [137]:
#
# Un poco de limpieza
#
!hdfs dfs -rm -r -f /tmp/wordcount
!hdfs dfs -rm /tmp/data.txt
!hdfs dfs -ls /tmp

Deleted /tmp/wordcount
Deleted /tmp/data.txt
